In [1]:
# Library Import

import os
import sys
%pwd

'f:\\My_Design\\Data_Science\\Data Science\\Data Science Notes\\Machine Learning\\Projects\\End-to-End Projects\\End-to-End-Machine-Learning-Project-with-Deployment\\research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'f:\\My_Design\\Data_Science\\Data Science\\Data Science Notes\\Machine Learning\\Projects\\End-to-End Projects\\End-to-End-Machine-Learning-Project-with-Deployment'

In [4]:
# Updating the Data Transformation Entity

# Perparing the data validation entities [config_entity]

from dataclasses import dataclass
from pathlib import Path

# Below is the data class for data Validation configuration
# This class is used to define the configuration for data Validation in a machine learning pipeline.
@dataclass(frozen=True)
class ModelTrainerConfig:
    """Model Training Configuration"""
    # Define the attributes of the class
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    random_state: int
    target_column: str

In [5]:
from ml_project.constants import *
from ml_project.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    """Configuration Manager"""
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        """Initialize Configuration Manager"""
        # Initialize the ConfigurationManager with file paths for configuration, parameters, and schema.
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)



        # Create directories for artifacts and data ingestion
        create_directories([self.config.artifacts_root])  # Create the artifacts root directory

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        """Get Model Trainer Configuration"""
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            random_state=params.random_state,
            target_column=schema.name,
        )
        
        return model_trainer_config

In [17]:
# COMPONENT 1: Model Trainer Component
# This component is responsible for training the machine learning model using the provided training data.

import os
from pathlib import Path
import pandas as pd
import numpy as np
from ml_project import logger
from sklearn.linear_model import ElasticNet
import joblib

In [ ]:
class ModelTrainer:
    """Model Trainer"""
    def __init__(self, config: ModelTrainerConfig):
        """Initialize Model Trainer"""
        self.config = config

    
    def train(self):
        """Train the model"""
        # Load the training data
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # Separate features and target variable from training and testing data
        X_train = train_data.drop(columns=[self.config.target_column], axis=1)
        y_train = train_data[self.config.target_column]
        X_test = test_data.drop(columns=[self.config.target_column], axis=1)
        y_test = test_data[self.config.target_column]

        # Initialize the ElasticNet model with hyperparameters
        lr = ElasticNet(
            alpha=self.config.alpha,
            l1_ratio=self.config.l1_ratio,
            random_state=self.config.random_state
        )

        # Train the model on the training data
        lr.fit(X_train, y_train)

        # Save the trained model to a file
        model_path = os.path.join(self.config.root_dir, self.config.model_name)
        # Save the model using joblib
        joblib.dump(lr, model_path)

        logger.info(f"Model trained and saved at: {model_path}")

In [22]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_training = ModelTrainer(model_trainer_config)
    model_training.train()
    print("Model training completed successfully.")
except Exception as e:
    raise e

[2025-05-07 00:49:23,500: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-07 00:49:23,503: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-07 00:49:23,508: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-07 00:49:23,511: INFO: common: created directory at: artifacts]
[2025-05-07 00:49:23,533: INFO: 2581246404: Model trained and saved at: artifacts/model_trainer\model.joblib]
Model training completed successfully.
